# Machine Learning Baselines for Hurst Exponent Estimation

**Author:** Davian R. Chin (PhD Candidate in Biomedical Engineering, University of Reading, UK)  
**Email:** d.r.chin@pgr.reading.ac.uk  
**ORCiD:** [https://orcid.org/0009-0003-9434-3919](https://orcid.org/0009-0003-9434-3919)  
**Research Focus:** Physics-Informed Fractional Operator Learning for Real-Time Neurological Biomarker Detection

---

## Overview

This comprehensive tutorial demonstrates how to use the Machine Learning baselines in the Neurological LRD Analysis library for accurate Hurst exponent estimation. We'll cover:

1. **Installation and Setup**
2. **Feature Extraction** (74+ features from time series)
3. **ML Baselines** (Random Forest, SVR, Gradient Boosting)
4. **Hyperparameter Optimization** (Optuna Integration)
5. **Pretrained Models** (Training, Storage, Loading)
6. **Inference Systems** (Single and Batch Predictions)
7. **Comprehensive Benchmarking** (Classical vs ML Methods)
8. **Performance Analysis** (Accuracy, Speed, Memory)

This notebook is designed for researchers and practitioners who want to leverage machine learning for improved Hurst exponent estimation in biomedical time series.


## 1. Installation and Setup


In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

# Import Neurological LRD Analysis library
from neurological_lrd_analysis import (
    # Classical methods for comparison
    BiomedicalHurstEstimatorFactory,
    EstimatorType,
    
    # ML baselines
    MLBaselineType,
    RandomForestEstimator,
    SVREstimator,
    GradientBoostingEstimator,
    MLBaselineFactory,
    TimeSeriesFeatureExtractor,
    
    # Hyperparameter optimization
    OptunaOptimizer,
    create_optuna_study,
    optimize_hyperparameters,
    optimize_all_estimators,
    
    # Pretrained models
    PretrainedModelManager,
    PretrainedInference,
    quick_predict,
    quick_ensemble_predict,
    create_pretrained_suite,
    
    # Benchmarking
    ClassicalMLBenchmark,
    run_comprehensive_benchmark
)

# Import data generation utilities
from neurological_lrd_analysis.benchmark_core.generation import (
    fbm_davies_harte,
    generate_fgn,
    generate_arfima,
    add_contamination
)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ All libraries imported successfully!")
print("🚀 Ready to explore ML baselines for Hurst estimation!")


## 2. Feature Extraction for Time Series

The first step in ML-based Hurst estimation is extracting meaningful features from time series data. Our library provides 74+ features across multiple categories.


In [ ]:
# Generate sample time series data for feature extraction
print("Generating sample time series data...")

# Create different types of time series with known Hurst exponents
hurst_values = [0.3, 0.5, 0.7, 0.9]
time_series_data = []
true_hurst_values = []

for hurst in hurst_values:
    # Generate FBM data
    fbm_data = fbm_davies_harte(1000, hurst, seed=42)
    time_series_data.append(fbm_data)
    true_hurst_values.append(hurst)
    
    # Generate FGN data
    fgn_data = generate_fgn(1000, hurst, seed=42)
    time_series_data.append(fgn_data)
    true_hurst_values.append(hurst)

print(f"Generated {len(time_series_data)} time series")
print(f"Hurst values range: {min(true_hurst_values):.1f} to {max(true_hurst_values):.1f}")

# Initialize feature extractor
feature_extractor = TimeSeriesFeatureExtractor()

# Extract features from first time series
sample_data = time_series_data[0]
print(f"\nExtracting features from time series of length {len(sample_data)}...")

# Extract all features
features = feature_extractor.extract_features(sample_data)
print(f"✅ Extracted {len(features)} features")

# Display feature categories
feature_categories = feature_extractor.get_feature_categories()
print(f"\nFeature categories available:")
for category, count in feature_categories.items():
    print(f"  - {category}: {count} features")


In [ ]:
# Extract features from all time series for ML training
print("Extracting features from all time series...")

X_features = []
y_targets = []

for i, (data, true_hurst) in enumerate(zip(time_series_data, true_hurst_values)):
    features = feature_extractor.extract_features(data)
    X_features.append(features)
    y_targets.append(true_hurst)
    
    if (i + 1) % 2 == 0:  # Print progress every 2 samples
        print(f"  Processed {i + 1}/{len(time_series_data)} time series")

# Convert to numpy arrays
X = np.array(X_features)
y = np.array(y_targets)

print(f"\n✅ Feature extraction complete!")
print(f"📊 Feature matrix shape: {X.shape}")
print(f"🎯 Target vector shape: {y.shape}")
print(f"📈 Hurst value range: {y.min():.2f} to {y.max():.2f}")

# Display some feature statistics
print(f"\nFeature statistics:")
print(f"  - Mean: {X.mean():.4f}")
print(f"  - Std: {X.std():.4f}")
print(f"  - Min: {X.min():.4f}")
print(f"  - Max: {X.max():.4f}")
print(f"  - NaN values: {np.isnan(X).sum()}")
print(f"  - Inf values: {np.isinf(X).sum()}")


## 3. Machine Learning Baselines

Now let's explore the three ML baselines: Random Forest, Support Vector Regression (SVR), and Gradient Boosting.


In [ ]:
# Split data for training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

# Initialize ML estimators
print("\nInitializing ML estimators...")

# Random Forest
rf_estimator = RandomForestEstimator(
    n_estimators=100,
    max_depth=10,
    random_state=42
)

# Support Vector Regression
svr_estimator = SVREstimator(
    C=1.0,
    gamma='scale',
    epsilon=0.1
)

# Gradient Boosting
gb_estimator = GradientBoostingEstimator(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

print("✅ All ML estimators initialized!")


In [ ]:
# Train all ML estimators
print("Training ML estimators...")

estimators = {
    'Random Forest': rf_estimator,
    'SVR': svr_estimator,
    'Gradient Boosting': gb_estimator
}

training_results = {}

for name, estimator in estimators.items():
    print(f"\nTraining {name}...")
    start_time = time.time()
    
    # Train the estimator
    result = estimator.train(X_train, y_train, validation_split=0.2)
    
    training_time = time.time() - start_time
    
    # Store results
    training_results[name] = {
        'result': result,
        'training_time': training_time,
        'estimator': estimator
    }
    
    print(f"  ✅ Training completed in {training_time:.2f} seconds")
    print(f"  📊 Training score: {result.training_score:.4f}")
    print(f"  📊 Validation score: {result.validation_score:.4f}")

print("\n🎉 All estimators trained successfully!")


In [ ]:
# Evaluate ML estimators on test set
print("Evaluating ML estimators on test set...")

test_results = {}

for name, data in training_results.items():
    estimator = data['estimator']
    
    # Make predictions
    start_time = time.time()
    predictions = estimator.predict(X_test)
    prediction_time = time.time() - start_time
    
    # Calculate metrics
    mse = np.mean((predictions - y_test) ** 2)
    mae = np.mean(np.abs(predictions - y_test))
    r2 = 1 - (np.sum((y_test - predictions) ** 2) / np.sum((y_test - np.mean(y_test)) ** 2))
    
    test_results[name] = {
        'predictions': predictions,
        'mse': mse,
        'mae': mae,
        'r2': r2,
        'prediction_time': prediction_time
    }
    
    print(f"\n{name} Performance:")
    print(f"  📊 MSE: {mse:.4f}")
    print(f"  📊 MAE: {mae:.4f}")
    print(f"  📊 R²: {r2:.4f}")
    print(f"  ⏱️  Prediction time: {prediction_time:.4f} seconds")

print("\n✅ All estimators evaluated!")


In [ ]:
# Visualize ML estimator performance
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: True vs Predicted values
ax1 = axes[0, 0]
for name, data in test_results.items():
    ax1.scatter(y_test, data['predictions'], alpha=0.7, label=name, s=50)
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', alpha=0.5)
ax1.set_xlabel('True Hurst Exponent')
ax1.set_ylabel('Predicted Hurst Exponent')
ax1.set_title('True vs Predicted Hurst Exponents')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Residuals
ax2 = axes[0, 1]
for name, data in test_results.items():
    residuals = y_test - data['predictions']
    ax2.scatter(data['predictions'], residuals, alpha=0.7, label=name, s=50)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)
ax2.set_xlabel('Predicted Hurst Exponent')
ax2.set_ylabel('Residuals')
ax2.set_title('Residuals vs Predicted Values')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Performance metrics comparison
ax3 = axes[1, 0]
metrics = ['MSE', 'MAE', 'R²']
x_pos = np.arange(len(metrics))
width = 0.25

for i, (name, data) in enumerate(test_results.items()):
    values = [data['mse'], data['mae'], data['r2']]
    ax3.bar(x_pos + i*width, values, width, label=name, alpha=0.8)

ax3.set_xlabel('Metrics')
ax3.set_ylabel('Values')
ax3.set_title('Performance Metrics Comparison')
ax3.set_xticks(x_pos + width)
ax3.set_xticklabels(metrics)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Prediction time comparison
ax4 = axes[1, 1]
names = list(test_results.keys())
times = [data['prediction_time'] for data in test_results.values()]
bars = ax4.bar(names, times, alpha=0.8, color=['skyblue', 'lightcoral', 'lightgreen'])
ax4.set_ylabel('Prediction Time (seconds)')
ax4.set_title('Prediction Speed Comparison')
ax4.grid(True, alpha=0.3)

# Add value labels on bars
for bar, time in zip(bars, times):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.0001, 
             f'{time:.4f}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Print summary table
print("\n📊 Performance Summary:")
print("=" * 80)
print(f"{'Estimator':<20} {'MSE':<10} {'MAE':<10} {'R²':<10} {'Time (s)':<10}")
print("-" * 80)
for name, data in test_results.items():
    print(f"{name:<20} {data['mse']:<10.4f} {data['mae']:<10.4f} {data['r2']:<10.4f} {data['prediction_time']:<10.4f}")


## 4. Hyperparameter Optimization with Optuna

Let's optimize the hyperparameters of our ML models using Optuna for better performance.


In [ ]:
# Optimize hyperparameters for all estimators
print("Optimizing hyperparameters with Optuna...")

# Optimize all estimators
optimization_results = optimize_all_estimators(
    X_train, y_train,
    n_trials=20,  # Reduced for demo
    validation_split=0.2,
    random_state=42
)

print("✅ Hyperparameter optimization completed!")
print(f"📊 Optimized {len(optimization_results)} estimators")

# Display optimization results
for estimator_type, result in optimization_results.items():
    print(f"\n{estimator_type}:")
    print(f"  🎯 Best score: {result.best_value:.4f}")
    print(f"  ⚙️  Best params: {result.best_params}")
    print(f"  🔄 Trials: {result.n_trials}")


## 5. Pretrained Models and Inference

Let's explore the pretrained model system for efficient model storage, loading, and inference.


In [ ]:
# Initialize pretrained model manager
import tempfile
import os

# Create temporary directory for models
temp_dir = tempfile.mkdtemp()
print(f"📁 Using temporary directory: {temp_dir}")

# Initialize pretrained model manager
model_manager = PretrainedModelManager(temp_dir)

# Create training data for pretrained models
print("Creating training data for pretrained models...")

X_pretrained, y_pretrained, training_info = model_manager.create_training_data(
    hurst_values=[0.3, 0.5, 0.7, 0.9],
    lengths=[500, 1000, 2000],
    n_samples_per_config=10,
    generators=['fbm', 'fgn'],
    contaminations=['none', 'noise'],
    biomedical_scenarios=['eeg', 'ecg']
)

print(f"✅ Training data created!")
print(f"📊 Features shape: {X_pretrained.shape}")
print(f"🎯 Targets shape: {y_pretrained.shape}")
print(f"📋 Training info: {len(training_info)} configurations")


In [ ]:
# Create pretrained model suite
print("Creating pretrained model suite...")

from neurological_lrd_analysis.ml_baselines.pretrained_models import (
    TrainingConfig,
    ModelStatus
)

# Define training configurations
training_configs = [
    TrainingConfig(
        model_type=MLBaselineType.RANDOM_FOREST,
        hyperparameters={'n_estimators': 100, 'max_depth': 10, 'random_state': 42},
        training_data_config={'n_samples_per_config': 10},
        description="Random Forest for Hurst estimation"
    ),
    TrainingConfig(
        model_type=MLBaselineType.SVR,
        hyperparameters={'C': 1.0, 'gamma': 'scale', 'epsilon': 0.1},
        training_data_config={'n_samples_per_config': 10},
        description="SVR for Hurst estimation"
    ),
    TrainingConfig(
        model_type=MLBaselineType.GRADIENT_BOOSTING,
        hyperparameters={'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3, 'random_state': 42},
        training_data_config={'n_samples_per_config': 10},
        description="Gradient Boosting for Hurst estimation"
    )
]

# Train the model suite
training_results = model_manager.create_model_suite(
    training_configs, X_pretrained, y_pretrained, training_info
)

print("✅ Pretrained model suite created!")
print(f"📊 Trained {len(training_results)} models")

# List available models
models = model_manager.list_models(status=ModelStatus.TRAINED)
print(f"📋 Available trained models: {len(models)}")
for model in models:
    print(f"  - {model.model_id}: {model.model_type} ({model.status})")


In [ ]:
# Test pretrained model inference
print("Testing pretrained model inference...")

# Initialize inference system
inference_system = PretrainedInference(temp_dir)

# Generate test data
test_data = fbm_davies_harte(1000, 0.6, seed=42)
test_features = feature_extractor.extract_features(test_data)

print(f"📊 Test data: {len(test_data)} samples")
print(f"🔍 Test features: {len(test_features)} features")

# Single prediction
print("\n🎯 Single Prediction:")
single_prediction = inference_system.predict_single(test_features)
print(f"  Predicted Hurst: {single_prediction:.4f}")

# Batch prediction
print("\n📦 Batch Prediction:")
batch_features = [feature_extractor.extract_features(fbm_davies_harte(1000, h, seed=42)) for h in [0.3, 0.5, 0.7, 0.9]]
batch_predictions = inference_system.predict_batch(batch_features)
print(f"  True Hurst: [0.3, 0.5, 0.7, 0.9]")
print(f"  Predicted: {[f'{p:.4f}' for p in batch_predictions]}")

# Ensemble prediction
print("\n🎭 Ensemble Prediction:")
ensemble_prediction = inference_system.predict_ensemble(test_features)
print(f"  Ensemble Hurst: {ensemble_prediction:.4f}")

# Quick prediction functions
print("\n⚡ Quick Predictions:")
quick_pred = quick_predict(test_features, temp_dir)
quick_ensemble = quick_ensemble_predict(test_features, temp_dir)
print(f"  Quick prediction: {quick_pred:.4f}")
print(f"  Quick ensemble: {quick_ensemble:.4f}")

print("\n✅ Pretrained model inference completed!")


## 6. Comprehensive Benchmarking: Classical vs ML Methods

Let's run a comprehensive benchmark comparing classical Hurst estimation methods with our ML baselines.


In [ ]:
# Initialize comprehensive benchmark
print("Initializing comprehensive benchmark...")

benchmark = ClassicalMLBenchmark(
    pretrained_models_dir=temp_dir,
    classical_estimators=[
        EstimatorType.DFA,
        EstimatorType.RS_ANALYSIS,
        EstimatorType.HIGUCHI,
        EstimatorType.GENERALIZED_HURST
    ],
    ml_estimators=[
        MLBaselineType.RANDOM_FOREST,
        MLBaselineType.SVR,
        MLBaselineType.GRADIENT_BOOSTING
    ],
    results_dir="./benchmark_results",
    random_state=42
)

print("✅ Benchmark initialized!")
print(f"🔬 Classical methods: {len(benchmark.classical_estimators)}")
print(f"🤖 ML methods: {len(benchmark.ml_estimators)}")

# Generate test scenarios
print("\nGenerating test scenarios...")
test_scenarios = []

# Generate diverse test scenarios
hurst_test_values = [0.3, 0.5, 0.7, 0.9]
for hurst in hurst_test_values:
    # FBM data
    fbm_data = fbm_davies_harte(1000, hurst, seed=42)
    from neurological_lrd_analysis.benchmark_core.generation import TimeSeriesSample
    sample = TimeSeriesSample(
        data=fbm_data,
        true_hurst=hurst,
        length=len(fbm_data),
        contamination='none',
        seed=42
    )
    test_scenarios.append(sample)
    
    # FGN data
    fgn_data = generate_fgn(1000, hurst, seed=42)
    sample = TimeSeriesSample(
        data=fgn_data,
        true_hurst=hurst,
        length=len(fgn_data),
        contamination='none',
        seed=42
    )
    test_scenarios.append(sample)

print(f"📊 Generated {len(test_scenarios)} test scenarios")
print(f"🎯 Hurst range: {min([s.true_hurst for s in test_scenarios]):.1f} to {max([s.true_hurst for s in test_scenarios]):.1f}")


In [ ]:
# Run comprehensive benchmark
print("Running comprehensive benchmark...")
print("=" * 60)

start_time = time.time()

# Run the benchmark
benchmark_results = benchmark.run_comprehensive_benchmark(
    test_scenarios=test_scenarios,
    save_results=True
)

benchmark_time = time.time() - start_time

print(f"\n✅ Benchmark completed in {benchmark_time:.2f} seconds!")
print(f"📊 Results saved to: {benchmark.results_dir}")

# Display benchmark summary
if 'summaries' in benchmark_results and benchmark_results['summaries']:
    print("\n📈 Benchmark Summary:")
    benchmark.print_benchmark_summary(benchmark_results['summaries'])
else:
    print("\n⚠️ No benchmark summaries available")


In [ ]:
# Visualize benchmark results
if 'summaries' in benchmark_results and benchmark_results['summaries']:
    print("Creating benchmark visualizations...")
    
    # Create visualizations
    benchmark.create_visualizations(
        benchmark_results, 
        output_path="./benchmark_plots.png"
    )
    
    print("✅ Visualizations created!")
    print("📊 Check './benchmark_plots.png' for detailed plots")
    
    # Display results table
    summaries = benchmark_results['summaries']
    
    print("\n📊 Detailed Results:")
    print("=" * 100)
    print(f"{'Method':<25} {'Type':<10} {'MAE':<10} {'RMSE':<10} {'Corr':<10} {'Time(ms)':<12} {'Success':<10}")
    print("-" * 100)
    
    for method_name, summary in summaries.items():
        print(f"{method_name:<25} {summary.method_type:<10} {summary.mean_absolute_error:<10.4f} "
              f"{summary.root_mean_squared_error:<10.4f} {summary.correlation:<10.4f} "
              f"{summary.mean_computation_time*1000:<12.1f} {summary.success_rate:<10.2f}")
    
    # Find best performers
    if summaries:
        best_mae = min(summaries.items(), key=lambda x: x[1].mean_absolute_error)
        best_speed = min(summaries.items(), key=lambda x: x[1].mean_computation_time)
        
        print(f"\n🏆 Best Performance:")
        print(f"  🎯 Lowest MAE: {best_mae[0]} ({best_mae[1].mean_absolute_error:.4f})")
        print(f"  ⚡ Fastest: {best_speed[0]} ({best_speed[1].mean_computation_time*1000:.1f}ms)")
        
        # Compare classical vs ML
        classical_methods = {k: v for k, v in summaries.items() if v.method_type == 'classical'}
        ml_methods = {k: v for k, v in summaries.items() if v.method_type == 'ml'}
        
        if classical_methods and ml_methods:
            best_classical = min(classical_methods.items(), key=lambda x: x[1].mean_absolute_error)
            best_ml = min(ml_methods.items(), key=lambda x: x[1].mean_absolute_error)
            
            print(f"\n🔬 Classical vs ML Comparison:")
            print(f"  🎯 Best Classical: {best_classical[0]} ({best_classical[1].mean_absolute_error:.4f})")
            print(f"  🤖 Best ML: {best_ml[0]} ({best_ml[1].mean_absolute_error:.4f})")
            
            if best_ml[1].mean_absolute_error < best_classical[1].mean_absolute_error:
                improvement = ((best_classical[1].mean_absolute_error - best_ml[1].mean_absolute_error) / 
                              best_classical[1].mean_absolute_error) * 100
                print(f"  🚀 ML improvement: {improvement:.1f}%")
            else:
                print(f"  📊 Classical methods perform better in this benchmark")

else:
    print("⚠️ No benchmark results available for visualization")


## 7. Summary and Conclusions

This tutorial demonstrated the comprehensive ML capabilities of the Neurological LRD Analysis library:

### 🎯 **Key Features Explored**

1. **Feature Extraction**: 74+ features across statistical, spectral, wavelet, and fractal categories
2. **ML Baselines**: Random Forest, SVR, and Gradient Boosting for Hurst estimation
3. **Hyperparameter Optimization**: Optuna integration for automatic parameter tuning
4. **Pretrained Models**: Efficient model storage, loading, and inference systems
5. **Comprehensive Benchmarking**: Direct comparison between classical and ML methods

### 📊 **Performance Insights**

- **ML methods** often provide more accurate Hurst estimates than classical methods
- **Feature extraction** is crucial for ML-based estimation
- **Hyperparameter optimization** can significantly improve model performance
- **Pretrained models** enable fast inference for production systems
- **Ensemble methods** can combine multiple models for robust predictions

### 🚀 **Next Steps**

1. **Experiment with different feature sets** for your specific applications
2. **Train custom models** on your domain-specific data
3. **Use pretrained models** for rapid prototyping and production deployment
4. **Run comprehensive benchmarks** to validate method performance on your data
5. **Explore ensemble methods** for improved accuracy and robustness

### 📚 **Additional Resources**

- **Documentation**: Check the library documentation for detailed API references
- **Examples**: Explore other notebooks for specific use cases
- **Benchmarks**: Run your own benchmarks with custom datasets
- **Contributions**: Contribute to the library development and improvement

---

**Happy analyzing! 🧠📈**
